<a href="https://colab.research.google.com/github/Rizkifadillah/googlecolab/blob/main/data_science/Unsupervised/Part_7_Document_Similarity_%26_Keyword_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Import Data

NLP

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/course_ipynb/Unstructured_Data/kompas.csv')

In [ ]:
df.sample(5)

,teks
1075,Argentina Menaklukkan Jerman\n\nJuara dunia du...
1903,Dalam Kasus Kecelakaan KA di Brebes Polisi P...
1369,"""Reserse"" Polda Metro Jaya Peras Empat Warga H..."
1382,"Perampok Berpistol Gasak Rp 2,5 Milyar\n\nLebi..."
1986,Malaysia Segera Deportasi 30.000 Imigran\n\nOt...


# Extract TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(df.teks)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


# TFIDF Similarity ---*untuk mendapatkan*--> Document Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [ ]:
# sorting
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]])

In [ ]:
df.teks[215]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pagi, mereka mengirimkan surat resmi kepada Kejagung, yang isinya meminta Kejagung untuk menghormati putusan pengadilan. Penahanan terhadap politisi Partai Golkar itu dinilai sudah tidak mempunyai landasan hukum.  Sebaliknya, Kepala Pusat Penerangan Hukum Kejaksaan Agung Moeljohardjo tetap berpendapat bahwa penahanan mantan Menteri Pertambangan dan Energi pada era Presiden Soeharto itu sah dan sesuai hukum. Dasar hukumnya adalah Surat Perintah Penahanan tanggal 31 Maret 2001 yang mengacu pada Undang-Undang Nomor 3 Tahun 1971 tentang Tindak Pidana Korupsi.  Hakim tunggal Rusman Dani Achmad dalam putusannya menyebutkan, menerima permohonan pemohon (Ginandjar) untuk sebagian. Dan kedua menyatakan, tindakan termohon (Jaksa Agung) melakukan penyidikan, penahanan, dan pembantaran dalam perkara Gi

In [ ]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [ ]:
df.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

# Keyword Extraction

In [ ]:
vocab = tfidf.get_feature_names_out()
vocab[-10:]

array(['zuniga memilih', 'zunnatul', 'zunnatul mafruhah', 'zurich',
       'zurich northholt', 'zw', 'zw suparman', 'zw tim', 'zx',
       'zx diserbu'], dtype=object)

In [ ]:
tfidf_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [21]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274212, 365469, 365468, ..., 386379, 436652, 169219])

In [22]:
vocab[169219]

'ginandjar'

In [23]:
vocab[436652]

'putusan'

In [24]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

In [29]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
  matrix = tfidf.transform([doc])
  vocab = tfidf.get_feature_names_out()

  sorted_tfidf = matrix[0].toarray()[0].argsort()
  return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]

In [27]:
text = """Pemuda asal Kendari, Sulawesi Tenggara, Benaia Manasye Lintjewas jadi sorotan usai dinyatakan lulus jadi tentara Amerika Serikat (AS). Benaia sendiri dikabarkan masih berstatus WNI.

Pria berusia 21 tahun ini iseng mendaftar menjadi tentara AS dan ternyata malah mendapatkan hasil nilai tertinggi di angkatannya. Berapa gaji Benaia jadi tentara AS?

Dilansir dari laman goarmy.com, Jumat (24/3/2023), gaji tentara AS dihitung berdasarkan formula kombinasi antara pangkat dan masa pengabdian. Semakin besar pangkatnya, maka akan semakin besar juga gajinya.

Benaia Manasye Lintjewas saat lulus jadi tentara AS./ Foto: Benaia Manasye Lintjewas saat lulus jadi tentara AS. Dokumen Istimewa
Bila tak kunjung naik pangkat pun gaji bakal naik dalam kurun waktu masa pengabdian. Ada 4 golongan masa pengabdian mulai dari kurang dari 2 tahun, 4 tahun, 6 tahun, dan 8 tahun.

Sementara pangkat di angkatan bersenjata di AS terbagi menjadi 3 golongan. Mulai dari golongan E atau Enlisted (Tamtama), W atau Warrant (Bintara Tinggi), O atau Officers (Perwira).

Baca Juga : Sederet Peraturan Unik bagi Pekerja dari Berbagai Negara, Ada yang Larang Lembur hingga Boleh Tidur Siang!
A U.S. army soldier stands guard in front of a F-22 stealth fighter jet (R) and F-15 fighter jet at Osan Air Base in Pyeongtaek, South Korea, February 17, 2016. REUTERS/Kim Hong-Ji/ Foto: REUTERS/Kim Hong-Ji
Untuk jabatan paling rendah, E1 alias Private dengan masa jabatan di bawah 2 tahun mendapatkan gaji tahunan sebesar US$ 23.011.

Bila dibagi selama 12 bulan maka per bulannya prajurit dengan jabatan E1 mendapatkan gaji sebesar US$ 1.917 atau sekitar Rp 29,4 juta (dalam kurs Rp 15.300).

Lalu berapa kenaikan yang didapat jika berhasil naik pangkat? Baca selengkapnya di sini, Beauties.

---

Ingin jadi salah satu pembaca yang bisa ikutan beragam event seru di Beautynesia? Yuk gabung ke komunitas pembaca Beautynesia B-Nation."""

In [30]:
extract_keywords_tfidf(text, tfidf)

['tentara as',
 'pangkat',
 'gaji',
 'tentara',
 'pengabdian',
 'lulus',
 'as',
 'us',
 'jet',
 'golongan']